
# imports

import os 
import requests 
from dotenv import load_dotenv 
from openai import OpenAI
from bs4 import BeautifulSoup
from IPython.display import Markdown , display

Creating a Parsing & Scrapping agent 

In [2]:
openai = OpenAI(base_url="http://localhost:11434/v1",api_key="ollama")


In [3]:
message = "Hello ollama this is first ever message to you "
response = openai.chat.completions.create(model="llama3.2",messages=[{"role":"user","content":message}])
print(response.choices[0].message.content)

It's great to be here and start our conversation. I'm OLLAMA, your friendly and knowledgeable AI assistant. I'm excited to help with any questions or topics you'd like to discuss.

I have to say, it's wonderful that this is the first-ever message to me too! A blank slate brings a lot of possibilities for interesting conversations. What would you like to talk about?


# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class website:
    def __init__(self,url):
        """create this website object from the given url using the beautifulsoup library 
        """
        self.url = url
        response = requests.get(url,headers)
        soup = BeautifulSoup(response.content,"html.parser")
        self.title = soup.title.string if soup.title else "No title Found"
        for irrelevant in soup.body(["scripts","style",'img',"input"]):
            irrelevant.decompose()
        self.txt = soup.body.get_text(separator="\n", strip = True)

In [13]:
user_url = input("Hi user can you enter the webiste from which you want the scarpping")
ari = website(user_url)
print(ari.title)
print(ari.txt)

Hi user can you enter the webiste from which you want the scarpping https://anthropic.com


Home \ Anthropic
Skip to main content
Skip to footer
Claude
Chat with Claude
Overview
Max plan
Team plan
Enterprise plan
Explore pricing
Download apps
Claude log in
News
Claude’s Character
API
Build with Claude
API overview
Developer docs
Explore pricing
Console log in
News
Learn how to build with Claude
Solutions
Collaborate with Claude
AI agents
Coding
Customer support
Education
Financial services
Government
Case studies
Hear from our customers
Research
Research
Overview
Economic Index
Claude model family
Claude Opus 4.1
Claude Sonnet 4
Claude Haiku 3.5
Research
Claude’s extended thinking
Commitments
Initiatives
Transparency
Responsible scaling policy
Trust center
Security and compliance
Announcement
ISO 42001 certification
Learn
Learning resources
Customer stories
Engineering at Anthropic
Anthropic Academy
Company
About
Careers
Events
Engineering
Building effective agents
News
EN
This is some text inside of a div block.
Try Claude
Try Claude
Try Claude
Download App
Download App
Down

In [37]:
response = openai.chat.completions.create(model = "llama3.2", messages = messages)
print(response.choices[0].message.content)

#Now trying by giving system prompt and user prompt
system_prompt = input("Enter the prompt")
print("\n The prompt is :\n",system_prompt)

In [22]:
def user_prompt_for(website):
    user_prompt = f"you are looking at a website titled {website.title}"
    user_prompt += "\n the contents of the website are as follows; \
    please provide a short summary of this website in markdown. \
    If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.txt
    return user_prompt 

    

In [23]:
print(user_prompt_for(ari))

you are looking at a website titled Home \ Anthropic
 the contents of the website are as follows;     please provide a short summary of this website in markdown.     If it includes news or announcements, then summarize these too.

Skip to main content
Skip to footer
Claude
Chat with Claude
Overview
Max plan
Team plan
Enterprise plan
Explore pricing
Download apps
Claude log in
News
Claude’s Character
API
Build with Claude
API overview
Developer docs
Explore pricing
Console log in
News
Learn how to build with Claude
Solutions
Collaborate with Claude
AI agents
Coding
Customer support
Education
Financial services
Government
Case studies
Hear from our customers
Research
Research
Overview
Economic Index
Claude model family
Claude Opus 4.1
Claude Sonnet 4
Claude Haiku 3.5
Research
Claude’s extended thinking
Commitments
Initiatives
Transparency
Responsible scaling policy
Trust center
Security and compliance
Announcement
ISO 42001 certification
Learn
Learning resources
Customer stories
Engineer

In [38]:
# messages = [
#     {"role": "system", "content": "You are a helpful assistant"},
#     {"role": "user", "content": "What is 2 + 2?"}
# ]


In [39]:
# response = openai.chat.completions.create(model = "llama3.2", messages = messages)
# print(response.choices[0].message.content)

2 + 2 = 4.


In [40]:
# now using the above syntax into our parsing agent 
def message_for(website):
    return[
        {"role":"system" ,"content":system_prompt},
        {"role":"user", "content":user_prompt_for(website)}
    ]

In [41]:
message_for(ari)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown."'},
 {'role': 'user',
  'content': 'you are looking at a website titled Home \\ Anthropic\n the contents of the website are as follows;     please provide a short summary of this website in markdown.     If it includes news or announcements, then summarize these too.\n\nSkip to main content\nSkip to footer\nClaude\nChat with Claude\nOverview\nMax plan\nTeam plan\nEnterprise plan\nExplore pricing\nDownload apps\nClaude log in\nNews\nClaude’s Character\nAPI\nBuild with Claude\nAPI\xa0overview\nDeveloper docs\nExplore pricing\nConsole log in\nNews\nLearn how to build with Claude\nSolutions\nCollaborate with Claude\nAI\xa0agents\nCoding\nCustomer support\nEducation\nFinancial services\nGovernment\nCase studies\nHear from our customers\nResearch\nResearch\nOverview\nEconomic Index\nClaude model famil

In [54]:
def summarize(url):
    site = website(url)  # or Website(url) if class is capitalized
    response = openai.chat.completions.create(
        model="llama3.2",
        messages=message_for(site)
    )
    return response.choices[0].message.content

In [55]:
summary = summarize(user_url) 